In [ ]:
'''Python Notebook for MIR Group Project'''
'''Jamila, Henry, and Paul'''
hi

In [5]:
import IPython.display as ipd
import librosa

srate, data = librosa.load('./mp3/Barrows Goldeneye2011-4-21-2.mp3')
ipd.Audio(data,rate=srate)

/Users/jamila/Library/Python/3.8/lib/python/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


ValueError: Array audio input must be a 1D or 2D array